In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

In [3]:
import sys

In [4]:
sys.path.append(".")

In [5]:
import ngm

## Read in metabric data from deeptype paper

In [6]:
import scipy.io
import numpy as np
import pandas as pd

mat = scipy.io.loadmat('../metabric/BRCA1View20000.mat')
data = mat['data']
mat = {k:v for k, v in mat.items() if k[0] != '_'}
print(len(mat['targets']))
#mat = {k: pd.Series(v[0]) for k, v in mat.items() if len(v) == 0}
mat = {k: pd.Series(v.flatten()) for k, v in mat.items()}

targets = pd.DataFrame.from_dict(mat).dropna()[['id', 'targets']].astype(int)

2133


In [7]:
data.shape

(20000, 2133)

In [8]:
X = data.T

In [9]:
X.shape

(2133, 20000)

## Read in our adjacency matrix

In [10]:
import pandas as pd
import io
import requests
url="https://metabric.s3-us-west-1.amazonaws.com/cat2vec/cat2vec_adj.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

In [11]:
c['id'] = c['Patient ID'].apply(lambda x: int(x.split("-")[1]))

In [12]:
c_merged = c.merge(targets, on='id')
c_merged = c_merged.set_index('Patient ID')
c_merged = c_merged[c_merged.index]

In [13]:
c_merged.shape

(494, 494)

In [14]:
c_merged

,MB-0002,MB-0005,MB-0006,MB-0008,MB-0010,MB-0014,MB-0020,MB-0022,MB-0028,MB-0035,...,MB-0885,MB-0891,MB-0893,MB-0895,MB-0897,MB-0899,MB-0901,MB-0904,MB-0906,MB-0907
Patient ID,,,,,,,,,,,,,,,,,,,,,
MB-0002,-1.000000,0.049814,0.294911,0.039187,-0.249322,0.348728,0.211491,0.074908,0.039009,-0.387732,...,0.074976,0.098055,0.481728,-0.627232,-0.322279,0.358629,0.242280,-0.341442,-0.358780,-0.151527
MB-0005,0.049814,-1.000000,-0.724841,-0.828614,0.055709,-0.629857,-0.549421,0.352115,0.405221,0.269612,...,0.342066,0.352254,-0.310700,-0.109071,0.054536,-0.644874,-0.529859,-0.003104,0.316354,-0.105369
MB-0006,0.294911,-0.724841,-1.000000,-0.655585,-0.232813,-0.822487,-0.300468,0.200873,0.301604,0.560098,...,0.228852,0.235086,-0.347182,0.200409,-0.191713,-0.839664,-0.288746,-0.200454,0.633718,-0.104822
MB-0008,0.039187,-0.828614,-0.655585,-1.000000,0.045452,-0.591871,-0.578162,0.348633,0.398693,0.225244,...,0.343301,0.370762,-0.302007,-0.150112,0.067975,-0.614083,-0.554857,0.028634,0.272523,-0.079890
MB-0010,-0.249322,0.055709,-0.232813,0.045452,-1.000000,-0.291201,0.505300,-0.075984,-0.028591,0.318952,...,-0.006957,-0.017840,0.288905,-0.023099,-0.729028,-0.248022,0.512799,-0.730339,0.355880,-0.159456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MB-0899,0.358629,-0.644874,-0.839664,-0.614083,-0.248022,-0.849860,-0.239179,0.217811,0.340402,0.652758,...,0.243182,0.256174,-0.329317,0.254777,-0.208431,-1.000000,-0.225838,-0.189629,0.685713,-0.122964
MB-0901,0.242280,-0.529859,-0.288746,-0.554857,0.512799,-0.200528,-0.870553,-0.111523,-0.103842,-0.245606,...,-0.151128,-0.122368,-0.695124,0.170607,0.480857,-0.225838,-1.000000,0.451450,-0.212358,0.270035
MB-0904,-0.341442,-0.003104,-0.200454,0.028634,-0.730339,-0.238811,0.476475,-0.087855,-0.068353,0.214131,...,-0.056034,-0.069930,0.282914,-0.109065,-0.744894,-0.189629,0.451450,-1.000000,0.240950,-0.108698


In [15]:
c_merged.index = list([int(c.split("-")[1]) for c in c_merged.index])
c_merged.columns = c_merged.index

In [16]:
c_merged

,2,5,6,8,10,14,20,22,28,35,...,885,891,893,895,897,899,901,904,906,907
2,-1.000000,0.049814,0.294911,0.039187,-0.249322,0.348728,0.211491,0.074908,0.039009,-0.387732,...,0.074976,0.098055,0.481728,-0.627232,-0.322279,0.358629,0.242280,-0.341442,-0.358780,-0.151527
5,0.049814,-1.000000,-0.724841,-0.828614,0.055709,-0.629857,-0.549421,0.352115,0.405221,0.269612,...,0.342066,0.352254,-0.310700,-0.109071,0.054536,-0.644874,-0.529859,-0.003104,0.316354,-0.105369
6,0.294911,-0.724841,-1.000000,-0.655585,-0.232813,-0.822487,-0.300468,0.200873,0.301604,0.560098,...,0.228852,0.235086,-0.347182,0.200409,-0.191713,-0.839664,-0.288746,-0.200454,0.633718,-0.104822
8,0.039187,-0.828614,-0.655585,-1.000000,0.045452,-0.591871,-0.578162,0.348633,0.398693,0.225244,...,0.343301,0.370762,-0.302007,-0.150112,0.067975,-0.614083,-0.554857,0.028634,0.272523,-0.079890
10,-0.249322,0.055709,-0.232813,0.045452,-1.000000,-0.291201,0.505300,-0.075984,-0.028591,0.318952,...,-0.006957,-0.017840,0.288905,-0.023099,-0.729028,-0.248022,0.512799,-0.730339,0.355880,-0.159456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899,0.358629,-0.644874,-0.839664,-0.614083,-0.248022,-0.849860,-0.239179,0.217811,0.340402,0.652758,...,0.243182,0.256174,-0.329317,0.254777,-0.208431,-1.000000,-0.225838,-0.189629,0.685713,-0.122964
901,0.242280,-0.529859,-0.288746,-0.554857,0.512799,-0.200528,-0.870553,-0.111523,-0.103842,-0.245606,...,-0.151128,-0.122368,-0.695124,0.170607,0.480857,-0.225838,-1.000000,0.451450,-0.212358,0.270035
904,-0.341442,-0.003104,-0.200454,0.028634,-0.730339,-0.238811,0.476475,-0.087855,-0.068353,0.214131,...,-0.056034,-0.069930,0.282914,-0.109065,-0.744894,-0.189629,0.451450,-1.000000,0.240950,-0.108698
906,-0.358780,0.316354,0.633718,0.272523,0.355880,0.660196,-0.188703,-0.357670,-0.469261,-0.792098,...,-0.431136,-0.419074,-0.091748,-0.147430,0.294370,0.685713,-0.212358,0.240950,-1.000000,0.234893


## Fix things so they aren't weird

In [17]:
mat.keys()

dict_keys(['GII', 'data', 'gene', 'id', 'targets'])

In [18]:
genes = [g[0] for g in mat['gene']]
pd.Series(genes)

0          SCGB2A2
1            MUCL1
2          SCGB1D2
3              PIP
4        LOC648852
           ...    
19995    LOC642453
19996    LOC652100
19997    LOC646050
19998    LOC644912
19999    LOC652294
Length: 20000, dtype: object

In [19]:
X_df = pd.DataFrame(X)
X_df.columns = genes
X_df = X_df.set_index(mat['id'])
X_df

,SCGB2A2,MUCL1,SCGB1D2,PIP,LOC648852,DNAJA2,TFF3,S100P,CPB1,CEACAM6,...,DLEU7,LOC641311,LOC645307,IL1RAP,LOC647149,LOC642453,LOC652100,LOC646050,LOC644912,LOC652294
2080,6.365374,5.610561,5.134799,5.735802,5.855315,5.443985,8.005882,5.615179,6.682096,5.756946,...,5.441128,5.409726,5.420998,5.396492,5.358751,5.424770,5.411055,5.242460,5.197767,5.643537
236,14.330442,7.979139,10.244240,7.138279,9.890661,11.195867,13.511164,10.548596,5.491578,6.805606,...,5.392657,5.213895,5.167652,5.273045,5.287980,5.398061,5.226934,5.396744,5.352158,5.811813
741,14.388331,6.016715,12.164435,5.272691,6.468386,5.265518,13.386034,8.764722,7.331663,11.967719,...,5.328035,5.707537,5.327874,5.549589,5.333051,5.368544,5.296182,5.694022,5.459744,5.269015
380,13.111243,7.251862,8.389121,8.664654,10.378585,5.544646,11.159766,8.263224,5.497906,5.923506,...,5.410289,5.351022,5.009093,5.345468,5.176322,5.326991,5.480259,5.292691,5.242160,5.635111
1446,12.243103,7.293489,10.919581,9.566137,8.503568,11.492583,11.531352,9.818938,9.443134,6.889670,...,5.402880,5.320883,5.487783,5.479967,5.332074,5.273141,5.282872,5.222482,5.186136,5.616094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880,7.199176,5.708891,5.672914,14.464282,9.816897,5.604835,13.303832,6.391817,13.010996,8.294409,...,5.398490,5.695733,5.363557,5.394837,5.442947,5.423763,5.558225,5.575000,5.636068,5.454823
858,10.354497,14.464282,7.633244,14.464282,11.294018,5.468214,10.886420,12.260033,5.662372,6.274380,...,5.378907,5.537254,5.613097,5.254552,5.205904,5.486546,5.206480,5.161718,5.451296,5.215587
698,9.294401,13.703049,6.063617,10.063176,5.393440,5.529584,10.937517,12.220400,9.985646,8.370227,...,5.554940,5.520596,5.572326,5.317225,5.178158,5.316757,5.210332,5.239428,5.395402,5.501265
1626,14.464282,13.203708,14.169804,13.731721,12.478907,5.347654,14.464282,9.593148,6.165212,11.207149,...,5.470530,5.650387,5.359733,5.374522,5.245155,5.453742,5.353135,5.235716,5.155543,5.442698


In [45]:
y = targets.set_index('id')['targets'].astype(int) # Richa: I wanted to change this to string but it broke pytroch
y = y - y.min() # make sure 0 is the first class. Richa: let's fix this as well
y.value_counts()

2    721
3    491
0    330
1    239
4    202
5    150
Name: targets, dtype: int64

In [46]:
A=c_merged

## Put data into the pytorch format

In [47]:
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

In [48]:
batch_size = 5       # The size of input data took for one iteration

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X_df.loc[A.index], y.loc[A.index], test_size=0.1, random_state=42)
X_train = torch.tensor(X_train.values).float()
y_train = torch.tensor(y_train.values).float()

In [50]:
X_test = torch.tensor(X_test.values).float()
y_test = torch.tensor(y_test.values).float()

In [51]:
dataset = ngm.base.MyDataset(X_train, y_train)
dataloader = DataLoader(dataset, batch_size=batch_size)
testset = ngm.base.MyDataset(X_test, y_test)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=True)

## Now construct the model

In [52]:
input_size = X_df.shape[1]
hidden_size = 50 # The number of nodes at the hidden layer
num_classes = len(y.unique())      # The number of output classes. In this case, from 1 to 39

In [53]:
model = ngm.base.Net(input_size, hidden_size, num_classes)

## Train the model

In [54]:
learning_rate = 0.001  # The speed of convergence
num_epochs=30

In [55]:
ngm.base.train(model,A,dataloader,dataset,learning_rate,num_epochs)

Epoch [1/30], Step [20/88], Loss: 3.7426
Epoch [1/30], Step [40/88], Loss: 1.8431
Epoch [1/30], Step [60/88], Loss: 1.8411
Epoch [1/30], Step [80/88], Loss: 1.8063
Epoch [2/30], Step [20/88], Loss: 1.7880
Epoch [2/30], Step [40/88], Loss: 1.7878
Epoch [2/30], Step [60/88], Loss: 1.7923
Epoch [2/30], Step [80/88], Loss: 1.7439
Epoch [3/30], Step [20/88], Loss: 1.7388
Epoch [3/30], Step [40/88], Loss: 1.7357
Epoch [3/30], Step [60/88], Loss: 1.7486
Epoch [3/30], Step [80/88], Loss: 1.6863
Epoch [4/30], Step [20/88], Loss: 1.6944
Epoch [4/30], Step [40/88], Loss: 1.6882
Epoch [4/30], Step [60/88], Loss: 1.7095
Epoch [4/30], Step [80/88], Loss: 1.6335
Epoch [5/30], Step [20/88], Loss: 1.6544
Epoch [5/30], Step [40/88], Loss: 1.6450
Epoch [5/30], Step [60/88], Loss: 1.6745
Epoch [5/30], Step [80/88], Loss: 1.5852
Epoch [6/30], Step [20/88], Loss: 1.6186
Epoch [6/30], Step [40/88], Loss: 1.6061
Epoch [6/30], Step [60/88], Loss: 1.6433
Epoch [6/30], Step [80/88], Loss: 1.5413
Epoch [7/30], St

## Evaluation

In [68]:
from sklearn.metrics import classification_report

In [69]:
ypred,ylabels = ngm.base.prepare_for_evaluation(model,testloader)

In [70]:
print(classification_report(ylabels, ypred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.00      0.00      0.00         6
           2       0.42      1.00      0.59        21
           3       0.00      0.00      0.00        13
           4       0.00      0.00      0.00         4

    accuracy                           0.42        50
   macro avg       0.08      0.20      0.12        50
weighted avg       0.18      0.42      0.25        50



In [73]:
pd.Series(ypred).value_counts()

2    50
dtype: int64